In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
%pip install --upgrade pip
%pip install boto3 --quiet
%pip install botocore --quiet
%pip install json --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [7]:
import base64
import boto3
import json
from IPython.display import Image



In [31]:
from llm_util import generate_conversation

def get_ocr(model_id, instructions_filename, input_image, temperature = 0):
    inference_config = {"temperature": temperature}
    instructions = open(f"./prompts/{instructions_filename}", "r").read()
    system_prompts = [{"text": instructions}]

    bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')
    response = generate_conversation(
        bedrock_client, model_id, system_prompts, inference_config, input_image)

    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")
    

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

    return output_message['content'][0]['text']

In [33]:
output = get_ocr(model_id = "anthropic.claude-3-sonnet-20240229-v1:0",
        input_image = "./images/pizza-1.png",
        instructions_filename = 'pizza_system_prompt1.txt',
        temperature = 0)

Generating message with model anthropic.claude-3-sonnet-20240229-v1:0
Role: assistant
Text: ```json
{
  "categories": [
    {
      "name": "פיצות חיות",
      "dishes": [
        {
          "name": "פיצה אישית (S)",
          "price": 20
        },
        {
          "name": "פיצה משפחתית (L)",
          "price": 29
        },
        {
          "name": "פיצה ענקית (XL)",
          "price": 49
        }
      ]
    },
    {
      "name": "מנות",
      "dishes": [
        {
          "name": "סטו דגרמית",
          "price": 34,
          "description": "בשר בקר, בצל, פטריות, שום, רוטב עגבניות ורוטב שמנת"
        },
        {
          "name": "סטו טבע",
          "price": 34,
          "description": "חצי, מלפפון, עגבניות, פטריות, שום, בצל, זיתים שחורים ירוקים"
        }
      ],
      "extraInfo": [
        "מוגש עם פיתה קלויה"
      ]
    },
    {
      "name": "תוספות",
      "dishes": [
        {
          "name": "שום, איולי, חמוצים",
          "price": 1
        },
        {
 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [42]:
import json_eval
filename = 'pizza_actual_' + model_id + f'_temp({temperature})_' + instructions_filename.replace('.txt', '') +'.json'
output_filename_fullpath = f'./outputs/{filename}'

print(f'Writing output to {output_filename_fullpath}')
open(output_filename_fullpath, 'w').write(output)

actual_json = json_eval.extract_json_from_string(output)

Writing output to ./outputs/pizza_actual_anthropic.claude-3-sonnet-20240229-v1:0_temp(0)_pizza_system_prompt1.json
{
  "categories": [
    {
      "name": "פיצות חיות",
      "dishes": [
        {
          "name": "פיצה אישית (S)",
          "price": 20
        },
        {
          "name": "פיצה משפחתית (L)",
          "price": 29
        },
        {
          "name": "פיצה ענקית (XL)",
          "price": 49
        }
      ]
    },
    {
      "name": "מנות",
      "dishes": [
        {
          "name": "סטו דגרמית",
          "price": 34,
          "description": "בשר בקר, בצל, פטריות, שום, רוטב עגבניות ורוטב שמנת"
        },
        {
          "name": "סטו טבע",
          "price": 34,
          "description": "חצי, מלפפון, עגבניות, פטריות, שום, בצל, זיתים שחורים ירוקים"
        }
      ],
      "extraInfo": [
        "מוגש עם פיתה קלויה"
      ]
    },
    {
      "name": "תוספות",
      "dishes": [
        {
          "name": "שום, איולי, חמוצים",
          "price": 1
       

In [44]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = actual_json

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 44.30%

Detailed Difference Report:
Differences found:
  Path: categories[0].extraInfo
  Type: Missing Key
  Value 1: ['תוספת שימושים בקצוות הפיצה 2 ₪']
  Value 2: Not present

  Path: categories[0].dishes[3].price
  Type: Value Difference
  Value 1: 40
  Value 2: 49

  Path: categories[0].dishes[3].name
  Type: String Difference
  Value 1: משפחתית טבעונית
  Value 2: פיצה ענקית (XL)

  Path: categories[0].dishes[3].description
  Type: Missing Key
  Value 1: V
  Value 2: Not present

  Path: categories[0].dishes[4].name
  Type: String Difference
  Value 1: משפחתית ללא גלוטן
  Value 2: פיצה ענקית (XL)

  Path: categories[0].dishes[4].price
  Type: Value Difference
  Value 1: 45
  Value 2: 49

  Path: categories[0].dishes[5].name
  Type: String Difference
  Value 1: המלצת השף פיצה יוונית
  Value 2: פיצה ענקית (XL)

  Path: categories[0].dishes[5].description
  Type: Missing Key
  Value 1: עגבניה,בצל,זיתים שחורים,בולגרית
  Value 2: Not present

  Path: categories[0].name


In [2]:
# Example usage
import json
json1 = json.loads('{"name": "John", "age": 30, "city": "New York", "hobbies": ["reading", "swimming"]}')
json2 = json.loads('{"name": "Jon", "age": 31, "city": "New York", "hobbies": ["reading", "running"]}')

from json_eval import compare_json

similarity_score, difference_report = compare_json(json1, json2)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 77.76%

Detailed Difference Report:
Differences found:
  Path: age
  Type: Value Difference
  Value 1: 30
  Value 2: 31

  Path: name
  Type: String Difference
  Value 1: John
  Value 2: Jon



In [28]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = json.load(open(output_filename_fullpath))

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 35.73%

Detailed Difference Report:
Differences found:
  Path: categories[0].extraInfo[0]
  Type: String Difference
  Value 1: תוספת שימושים בקצוות הפיצה 2 ₪
  Value 2: תוספת ביצה קשה לסלט 3 ש"ח

  Path: categories[0].dishes[0].price
  Type: Value Difference
  Value 1: 20
  Value 2: 34

  Path: categories[0].dishes[0].name
  Type: String Difference
  Value 1: פיצה אישית (S)
  Value 2: סלט טורקי

  Path: categories[0].dishes[0].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וזיתים שרויים

  Path: categories[0].dishes[1].price
  Type: Value Difference
  Value 1: 29
  Value 2: 34

  Path: categories[0].dishes[1].name
  Type: String Difference
  Value 1: פיצה משפחתית (L)
  Value 2: סלט בגרוזית

  Path: categories[0].dishes[1].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וביצים שרויים

  Path: categories[0].dishes[2].price
  Type: Value Difference
  Value 1: 49
  Value 2